<a href="https://colab.research.google.com/github/iitjranjeet/Generative_AI_LLM/blob/main/Fine-tuning%20Mistral%207b%20using%20LoRA%20technique%20on%20Own%20Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Fine-tuning Mistral 7b using LoRA technique on Own Dataset


Setup Runtime
For fine-tuning Llama, a GPU instance is essential. Follow the directions below:

- Go to `Runtime` (located in the top menu bar).
- Select `Change Runtime Type`.
- Choose `T4 GPU` (or a comparable option).

### Step 1: Setup Environment

In [1]:
!pip install pandas autotrain-advanced -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 38.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.2/78.2 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.6/51.6 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.0/66.0 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.3/341.3 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 64.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.9/269.9 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.5/225.5 kB 18.9 MB/s eta 0:0

In [2]:
!autotrain setup --update-torch

INFO     | 2025-01-25 14:13:05 | autotrain.cli.run_setup:run:43 - Installing latest xformers
INFO     | 2025-01-25 14:13:06 | autotrain.cli.run_setup:run:45 - Successfully installed latest xformers
INFO     | 2025-01-25 14:13:06 | autotrain.cli.run_setup:run:51 - Installing latest PyTorch
INFO     | 2025-01-25 14:13:08 | autotrain.cli.run_setup:run:53 - Successfully installed latest PyTorch


## Step 2: Connect to HuggingFace for Model Upload

### Logging to Hugging Face
To make sure the model can be uploaded to be used for Inference, it's necessary to log in to the Hugging Face hub.

### Getting a Hugging Face token
Steps:

1. Navigate to this URL: https://huggingface.co/settings/tokens
2. Create a write `token` and copy it to your clipboard
3. Run the code below and enter your `token`

In [3]:
from huggingface_hub import notebook_login
notebook_login()

## Step 3: Upload your dataset

Add your data set to the root directory in the Colab under the name train.csv. The AutoTrain command will look for your data there under that name.

#### Don't have a data set and want to try finetuning on an example data set?
If you don't have a dataset you can run these commands below to get an example data set and save it to train.csv

In [4]:
!git clone https://github.com/joshbickett/finetune-llama-2.git
%cd finetune-llama-2
%mv train.csv ../train.csv
%cd ..

Cloning into 'finetune-llama-2'...
remote: Enumerating objects: 70, done.
remote: Counting objects: 100% (70/70), done.
remote: Compressing objects: 100% (50/50), done.
remote: Total 70 (delta 38), reused 48 (delta 19), pack-reused 0 (from 0)
Receiving objects: 100% (70/70), 25.13 KiB | 12.57 MiB/s, done.
Resolving deltas: 100% (38/38), done.
/content/finetune-llama-2
/content


In [5]:
import pandas as pd
df = pd.read_csv("train.csv")
df

,Concept,Funny Description Prompt,text
0,A cactus at a dance party,"A cactus, wearing disco lights and surrounded ...",###Human:\nGenerate a midjourney prompt for A ...
1,A robot on a first date,"A robot, with a bouquet of USB cables, nervous...",###Human:\nGenerate a midjourney prompt for A ...
2,A snail at a speed contest,"A snail, with a mini rocket booster, confident...",###Human:\nGenerate a midjourney prompt for A ...
3,A penguin at a beach party,"A penguin, with sunscreen and a surfboard, try...",###Human:\nGenerate a midjourney prompt for A ...
4,A cloud in a bad mood,"A cloud, grumbling and dropping mini lightning...",###Human:\nGenerate a midjourney prompt for A ...
...,...,...,...
112,A donut feeling the hole emptiness,"A donut, in existential bakery therapy, ponder...",###Human:\nGenerate a midjourney prompt for A ...
113,A pineapple with a prickly attitude,"A pineapple, in a prickly personality class, s...",###Human:\nGenerate a midjourney prompt for A ...
114,A calculator crunching life's problems,"A calculator, at a problem-solving workshop, c...",###Human:\nGenerate a midjourney prompt for A ...
115,A kite reaching new heights,"A kite, in an altitude adjustment session, unt...",###Human:\nGenerate a midjourney prompt for A ...


In [6]:
df['text'][15]

'###Human:\nGenerate a midjourney prompt for A book on a mystery adventure\n\n###Assistant:\nA book, wearing detective glasses, flipping through its own pages, trying to solve the cliffhanger it was left on.'

In [10]:
!pip install --upgrade autotrain-advanced

In [24]:
from transformers import TrainingArguments, Trainer
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import BitsAndBytesConfig
from peft import LoraConfig, get_peft_model
import torch

# Define model configurations and parameters
# Changed model_name to the correct identifier on Hugging Face Model Hub
model_name = "mistralai/Mistral-7B-Instruct-v0.1"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# Create PEFT configuration
peft_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

# Load and wrap model with PEFT
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto"
)
model = get_peft_model(model, peft_config)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

NameError: name 'your_train_dataset' is not defined

In [33]:
training_args = TrainingArguments(
    output_dir="./mistral-finetuned",
    per_device_train_batch_size=1,  # Adjust as needed
    gradient_accumulation_steps=4,  # Adjust as needed
    num_train_epochs=3,
    fp16=True,
    logging_dir="./logs",
    learning_rate=2e-4,
    save_strategy="epoch",
    push_to_hub=True,
    report_to="none",
    hub_model_id="IITJ-Ranjeet/mistral-7b-finetuned_ranjeet",  # Update with your Hub model ID
    # The 'compute_metrics' argument is now supported in the updated transformers version
    # evaluation_strategy="epoch", # Uncomment if you want to evaluate at the end of each epoch
    metric_for_best_model="loss", # Add this line to specify the metric for the best model, this is essential when evaluation_strategy is set to "epoch"
    greater_is_better=False, #Since we want to minimize the loss function
)


In [41]:
from datasets import load_dataset, Dataset
df = pd.read_csv("train.csv")

# Convert the DataFrame to a Hugging Face Dataset
train_dataset = Dataset.from_pandas(df)

# Define a function to tokenize your data
def tokenize_function(examples):
    # Assuming 'Funny Description Prompt' column contains the input text
    # and 'text' column contains the target text
    # Adjust column names if necessary
    # Explicitly set max_length for padding and truncation
    tokenizer.pad_token = tokenizer.eos_token
    inputs = tokenizer(examples["Funny Description Prompt"], padding="max_length", truncation=True, max_length=128, return_tensors="pt")

    # Changed "response" to "text" to match the actual column name
    targets = tokenizer(examples["text"], padding="max_length", truncation=True, max_length=128, return_tensors="pt")

    inputs["labels"] = targets["input_ids"].clone()  # Set labels for loss computation
    return inputs

# Apply the tokenization function to your dataset
tokenized_datasets = train_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/117 [00:00<?, ? examples/s]

In [42]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
)
trainer.train()

# Save and push to Hugging Face Hub
trainer.save_model()
trainer.push_to_hub()

Step,Training Loss


No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/IITJ-Ranjeet/mistral-7b-finetuned_ranjeet/commit/432ca78cb013961025fb79530e9bcee15d9f0c4b', commit_message='End of training', commit_description='', oid='432ca78cb013961025fb79530e9bcee15d9f0c4b', pr_url=None, repo_url=RepoUrl('https://huggingface.co/IITJ-Ranjeet/mistral-7b-finetuned_ranjeet', endpoint='https://huggingface.co', repo_type='model', repo_id='IITJ-Ranjeet/mistral-7b-finetuned_ranjeet'), pr_revision=None, pr_num=None)

## Step 6: Inference Engine

In [43]:
!pip install -q peft  accelerate bitsandbytes safetensors

In [46]:
import torch
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer
import transformers
adapters_name = "IITJ-Ranjeet/mistral-7b-finetuned_ranjeet"
model_name = "mistralai/Mistral-7B-Instruct-v0.1" #"mistralai/Mistral-7B-Instruct-v0.1"


device = "cuda" # the device to load the model onto

In [47]:
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [49]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    # load_in_4bit=True,
    torch_dtype=torch.bfloat16,
    quantization_config=bnb_config,
    device_map='auto'
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

## Step 7: Peft Model Loading with upload model

In [50]:
model = PeftModel.from_pretrained(model, adapters_name)

adapter_config.json:   0%|          | 0.00/730 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/13.6M [00:00<?, ?B/s]

In [51]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.bos_token_id = 1

stop_token_ids = [0]

print(f"Successfully loaded the model {model_name} into memory")

Successfully loaded the model mistralai/Mistral-7B-Instruct-v0.1 into memory


In [53]:
text = "[INST] generate a midjourney prompt for A person walks in the rain [/INST]"

encoded = tokenizer(text, return_tensors="pt", add_special_tokens=False)
model_input = encoded.to(device)
model.to(device)
generated_ids = model.generate(**model_input, max_new_tokens=200, do_sample=True)
decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[INST] generate a midjourney prompt for A person walks in the rain [/INST] A person stands beneath a dark cloud, feeling the droplets of rain splattering upon their skin. Suddenly, the person decides to let go of their worries and let the rain soak their troubles away. They start walking, letting the rain pour over them and wash away their stress. As they walks, the sky begins to brighten and a rainbow appears in the distance. The person follows the rainbow, hoping to find a bright future waiting for them. They continue to walk, embracing the rain and the new opportunities that come with it. This journey of self-discovery and growth reminds them that even in the darkest moments, there is always a chance to find hope and a brighter tomorrow.</s>
